In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import load_model

2026-01-13 03:25:17.136048: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-13 03:25:17.181168: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-01-13 03:25:17.181206: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-01-13 03:25:17.182536: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-01-13 03:25:17.189426: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-13 03:25:17.190717: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [4]:
## load the imdb dataset word index
word_index = imdb.get_word_index()
reversed_word_index = {value: key for key, value in word_index.items()}

In [8]:
## Load the model

model = load_model('simple_rnn_model.h5')
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_2 (SimpleRNN)        │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,027 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [7]:
model.get_weights()

[array([[-0.328359  , -0.26759115, -0.10073621, ..., -0.13989131,
         -0.07385935, -0.02031984],
        [-0.11550557, -0.08237072,  0.04993395, ...,  0.02263232,
          0.00804259,  0.05546771],
        [-0.20850515, -0.0879485 ,  0.0020033 , ..., -0.06030046,
         -0.1167897 , -0.0550359 ],
        ...,
        [ 0.04859032,  0.11819252,  0.00180343, ..., -0.06178113,
         -0.08011969,  0.03046143],
        [ 0.16407506,  0.14749067, -0.13880228, ..., -0.09847558,
          0.10032079, -0.0035004 ],
        [-0.24558175, -0.23656043,  0.2041511 , ...,  0.04454099,
         -0.11006504,  0.14084503]], dtype=float32),
 array([[ 1.1360889e-01,  4.6809040e-02,  1.2816028e-01, ...,
          1.3128661e-01, -1.4308149e-01,  3.9795619e-02],
        [ 1.3882955e-04, -3.9533181e-05, -8.3546862e-02, ...,
          1.8722193e-02, -1.2451280e-01,  1.2788756e-01],
        [-6.2613808e-02, -3.4673825e-02,  1.3241290e-01, ...,
         -1.1366389e-01,  1.3812321e-01, -4.7887791e-02]

In [20]:
## Functions to decode review and pre-preocess the text\
from tensorflow.keras.preprocessing import sequence

def decode_review(encoded_review):
    return ' '.join([reversed_word_index.get(i - 3, '?') for i in encoded_review])

def preprocess(text):
    words = text.lower().split()
    encoded_review = [word_index.get(word, 2) + 3 for word in words]
    padded_review = sequence.pad_sequences([encoded_review], maxlen=500)
    return padded_review

In [21]:
## Predict function

def predict(review):
    preprocessed_input = preprocess(review)

    prediction = model.predict(preprocessed_input)

    sentiment = 'Positive' if prediction[0][0] > 0.5 else 'Negative'

    return sentiment, prediction[0][0]


In [27]:
## take user input for prediction

example_review = "The movie was awesome. The acting was good"

senttiment,score = predict(example_review)
senttiment, score

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step


('Positive', 0.9539949)